# Module 06: Volatility Mathematics

**Difficulty**: ⭐⭐ (Intermediate)

**Estimated Time**: 60 minutes

**Prerequisites**: 
- Module 00: Introduction and Stock Returns
- Module 01: Averages and Central Tendency
- Module 02: Spread and Variation (standard deviation)
- Module 03: Percentages, Ratios, and Changes
- Module 04: Moving Averages

## Learning Objectives

By the end of this notebook, you will be able to:
1. Calculate **Bollinger Bands** with complete statistical foundation
2. Master **Average True Range (ATR)** for volatility measurement
3. Build **Keltner Channels** using ATR
4. Detect **volatility squeeze** and **expansion** patterns
5. Use volatility for **position sizing** and **stop-loss placement**
6. Compare different volatility indicators mathematically

## Why This Matters

**Volatility = Risk = Opportunity**

Volatility indicators answer critical questions:
- **How much risk am I taking?** → Use ATR for position sizing
- **Where should I place my stop-loss?** → Use ATR multiples
- **Is price range expanding or contracting?** → Bollinger Bands width
- **Is a breakout coming?** → Volatility squeeze detection
- **Are we in trending or ranging market?** → Compare ATR levels

Applications:
- **Risk management** - Size positions based on volatility
- **Stop-loss placement** - Adaptive stops using ATR
- **Breakout trading** - Identify squeeze → expansion cycles
- **Mean reversion** - Trade extremes using Bollinger Bands
- **Trend confirmation** - Expanding volatility confirms trends

---

## Setup

In [ ]:
# Data manipulation and numerical operations
import pandas as pd
import numpy as np

# Data acquisition
import yfinance as yf

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Display settings
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Pandas display options
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)
pd.set_option('display.precision', 4)

# Random seed
np.random.seed(42)

print("✓ Libraries imported successfully!")

In [ ]:
# Download Malaysian stock data
print("Downloading Malaysian stock data...\n")

# Maybank - stable for testing Bollinger Bands
maybank = yf.download('1155.KL', start='2023-01-01', end='2024-01-01', progress=False)

# Top Glove - volatile for testing ATR
topglove = yf.download('5225.KL', start='2023-01-01', end='2024-01-01', progress=False)

# CIMB - moderate volatility
cimb = yf.download('1023.KL', start='2023-01-01', end='2024-01-01', progress=False)

# Validate
assert len(maybank) > 0, "Failed to download Maybank data"
assert len(topglove) > 0, "Failed to download Top Glove data"
assert len(cimb) > 0, "Failed to download CIMB data"

print(f"✓ Maybank: {len(maybank)} days")
print(f"✓ Top Glove: {len(topglove)} days")
print(f"✓ CIMB: {len(cimb)} days")
print("\nData ready!")

---

## Part 1: Bollinger Bands - Complete Analysis

### What are Bollinger Bands?

**Bollinger Bands** use statistical standard deviation to create dynamic support and resistance.

### Formula (Detailed)

$$
\begin{aligned}
\text{Middle Band} &= SMA_n(\text{Close}) \\
\text{Upper Band} &= SMA_n + (k \times \sigma_n) \\
\text{Lower Band} &= SMA_n - (k \times \sigma_n)
\end{aligned}
$$

Where:
- $SMA_n$ = Simple Moving Average (typically 20 periods)
- $\sigma_n$ = Standard deviation over n periods
- $k$ = Multiplier (typically 2 for 95% confidence)

### Statistical Foundation

Based on the **68-95-99.7 rule** (from Module 02):
- 1 std dev = ~68% of data
- **2 std dev = ~95% of data** ← Standard Bollinger Bands
- 3 std dev = ~99.7% of data

This means price should stay within the bands ~95% of the time!

In [ ]:
def calculate_bollinger_bands(prices, period=20, num_std=2):
    """
    Calculate Bollinger Bands.
    
    Parameters:
    -----------
    prices : pd.Series
        Price data
    period : int
        SMA period (default 20)
    num_std : float
        Number of standard deviations (default 2)
    
    Returns:
    --------
    tuple: (middle_band, upper_band, lower_band, bandwidth, %B)
    """
    # Calculate middle band (SMA)
    middle_band = prices.rolling(window=period).mean()
    
    # Calculate standard deviation
    std_dev = prices.rolling(window=period).std()
    
    # Calculate upper and lower bands
    upper_band = middle_band + (num_std * std_dev)
    lower_band = middle_band - (num_std * std_dev)
    
    # Calculate bandwidth (measure of volatility)
    bandwidth = ((upper_band - lower_band) / middle_band) * 100
    
    # Calculate %B (price position within bands)
    percent_b = (prices - lower_band) / (upper_band - lower_band)
    
    return middle_band, upper_band, lower_band, bandwidth, percent_b

# Calculate Bollinger Bands for Maybank
maybank_close = maybank['Close']

bb_middle, bb_upper, bb_lower, bb_width, bb_pct_b = calculate_bollinger_bands(
    maybank_close, period=20, num_std=2
)

print("Bollinger Bands for Maybank (20, 2)\n")
print("=" * 70)
print(f"\nCurrent Values:")
print(f"  Price:       RM {maybank_close.iloc[-1]:.4f}")
print(f"  Upper Band:  RM {bb_upper.iloc[-1]:.4f} (+{bb_upper.iloc[-1] - bb_middle.iloc[-1]:.4f})")
print(f"  Middle Band: RM {bb_middle.iloc[-1]:.4f}")
print(f"  Lower Band:  RM {bb_lower.iloc[-1]:.4f} ({bb_lower.iloc[-1] - bb_middle.iloc[-1]:.4f})")
print(f"\n  Bandwidth:   {bb_width.iloc[-1]:.2f}%")
print(f"  %B:          {bb_pct_b.iloc[-1]:.4f}")

# Interpret %B
current_b = bb_pct_b.iloc[-1]
if current_b > 1:
    print(f"\n  → Price above upper band (overbought)")
elif current_b < 0:
    print(f"\n  → Price below lower band (oversold)")
elif current_b > 0.8:
    print(f"\n  → Price near upper band (strong)")
elif current_b < 0.2:
    print(f"\n  → Price near lower band (weak)")
else:
    print(f"\n  → Price in middle zone (neutral)")

In [ ]:
# Visualize Bollinger Bands with all components
fig, axes = plt.subplots(3, 1, figsize=(14, 12), height_ratios=[2, 1, 1])

# Price with Bollinger Bands
axes[0].plot(maybank_close.index, maybank_close.values,
            linewidth=1.5, label='Price', color='black', alpha=0.8, zorder=5)
axes[0].plot(bb_middle.index, bb_middle.values,
            linewidth=2, label='Middle (SMA 20)', color='blue', linestyle='--')
axes[0].plot(bb_upper.index, bb_upper.values,
            linewidth=1.5, label='Upper (+2σ)', color='red', linestyle=':')
axes[0].plot(bb_lower.index, bb_lower.values,
            linewidth=1.5, label='Lower (-2σ)', color='green', linestyle=':')
axes[0].fill_between(bb_middle.index, bb_lower, bb_upper, alpha=0.1, color='gray')

# Mark when price touches bands
touch_upper = maybank_close > bb_upper
touch_lower = maybank_close < bb_lower
axes[0].scatter(maybank_close[touch_upper].index, maybank_close[touch_upper].values,
               color='red', s=30, alpha=0.6, zorder=6, label='Upper band touch')
axes[0].scatter(maybank_close[touch_lower].index, maybank_close[touch_lower].values,
               color='green', s=30, alpha=0.6, zorder=6, label='Lower band touch')

axes[0].set_title('Maybank - Bollinger Bands (20, 2)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Price (RM)')
axes[0].legend(loc='best', fontsize=9)
axes[0].grid(True, alpha=0.3)

# Bandwidth (volatility indicator)
axes[1].plot(bb_width.index, bb_width.values,
            linewidth=2, color='purple', alpha=0.8)
axes[1].axhline(y=bb_width.mean(), color='orange', linestyle='--',
               linewidth=2, label=f'Average ({bb_width.mean():.2f}%)')
axes[1].fill_between(bb_width.index, 0, bb_width.values,
                     where=(bb_width < bb_width.quantile(0.2)),
                     alpha=0.3, color='red', label='Squeeze (low volatility)')
axes[1].fill_between(bb_width.index, 0, bb_width.values,
                     where=(bb_width > bb_width.quantile(0.8)),
                     alpha=0.3, color='green', label='Expansion (high volatility)')
axes[1].set_title('Bandwidth (Volatility Measure)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Bandwidth (%)')
axes[1].legend(loc='best', fontsize=9)
axes[1].grid(True, alpha=0.3)

# %B (position within bands)
axes[2].plot(bb_pct_b.index, bb_pct_b.values,
            linewidth=2, color='blue', alpha=0.8)
axes[2].axhline(y=1, color='red', linestyle='--', linewidth=1, alpha=0.7, label='Upper band')
axes[2].axhline(y=0, color='green', linestyle='--', linewidth=1, alpha=0.7, label='Lower band')
axes[2].axhline(y=0.5, color='gray', linestyle=':', linewidth=1, alpha=0.5, label='Middle')
axes[2].fill_between(bb_pct_b.index, 0.8, 1.2, alpha=0.1, color='red')
axes[2].fill_between(bb_pct_b.index, -0.2, 0.2, alpha=0.1, color='green')
axes[2].set_title('%B (Price Position)', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Date')
axes[2].set_ylabel('%B')
axes[2].set_ylim(-0.2, 1.2)
axes[2].legend(loc='best', fontsize=9)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistics
print(f"\nBollinger Band Statistics (2023):")
print(f"  Days above upper band: {touch_upper.sum()} ({touch_upper.sum()/len(maybank_close)*100:.1f}%)")
print(f"  Days below lower band: {touch_lower.sum()} ({touch_lower.sum()/len(maybank_close)*100:.1f}%)")
print(f"  Days within bands: {(~touch_upper & ~touch_lower).sum()} ({(~touch_upper & ~touch_lower).sum()/len(maybank_close)*100:.1f}%)")
print(f"\n  Expected within bands: ~95%")
print(f"  Actual: {(~touch_upper & ~touch_lower).sum()/len(maybank_close)*100:.1f}%")
print(f"\n  Average bandwidth: {bb_width.mean():.2f}%")
print(f"  Min bandwidth (squeeze): {bb_width.min():.2f}%")
print(f"  Max bandwidth (expansion): {bb_width.max():.2f}%")

### Trading with Bollinger Bands

**Mean Reversion Strategy:**
1. Price touches lower band → Potential buy (oversold)
2. Price touches upper band → Potential sell (overbought)
3. Wait for price to move back toward middle

**Trend Following Strategy:**
1. Price rides upper band → Strong uptrend ("walking the band")
2. Price rides lower band → Strong downtrend
3. Don't fade the trend!

**Squeeze Strategy:**
1. Bandwidth contracts (squeeze) → Low volatility
2. Wait for expansion → Breakout coming
3. Trade in direction of breakout

---

## Part 2: Average True Range (ATR) - Pure Volatility

### What is ATR?

**Average True Range** measures the actual price movement, including gaps.

### Why ATR vs Simple Range?

**Problem with simple range (High - Low):**
- Ignores overnight gaps
- Doesn't capture full volatility

**ATR solves this** by considering gaps!

### Formula

**Step 1: Calculate True Range (TR)**

$$
TR = \max\begin{cases}
\text{High} - \text{Low} \\
|\text{High} - \text{Close}_{\text{previous}}| \\
|\text{Low} - \text{Close}_{\text{previous}}|
\end{cases}
$$

**Step 2: Calculate ATR (Wilder's smoothing)**

$$
ATR_n = \text{EMA}_{n}(TR)
$$

Typically n = 14 periods.

In [ ]:
def calculate_atr(high, low, close, period=14):
    """
    Calculate Average True Range (ATR).
    
    Parameters:
    -----------
    high : pd.Series
        High prices
    low : pd.Series
        Low prices
    close : pd.Series
        Close prices
    period : int
        ATR period (default 14)
    
    Returns:
    --------
    tuple: (tr, atr)
    """
    # Calculate True Range components
    tr1 = high - low  # Current day range
    tr2 = (high - close.shift(1)).abs()  # Gap up from previous close
    tr3 = (low - close.shift(1)).abs()   # Gap down from previous close
    
    # True Range = maximum of the three
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    
    # ATR = Wilder's smoothing (EMA with alpha = 1/period)
    atr = tr.ewm(alpha=1/period, adjust=False).mean()
    
    return tr, atr

# Calculate ATR for Top Glove (volatile stock)
tg_high = topglove['High']
tg_low = topglove['Low']
tg_close = topglove['Close']

tr, atr = calculate_atr(tg_high, tg_low, tg_close, period=14)

# Also calculate simple range for comparison
simple_range = tg_high - tg_low

print("Average True Range (ATR) for Top Glove\n")
print("=" * 70)
print(f"\nCurrent Values:")
print(f"  Price:        RM {tg_close.iloc[-1]:.4f}")
print(f"  True Range:   RM {tr.iloc[-1]:.4f}")
print(f"  ATR (14):     RM {atr.iloc[-1]:.4f}")
print(f"  Simple Range: RM {simple_range.iloc[-1]:.4f}")
print(f"\nATR as % of price: {(atr.iloc[-1] / tg_close.iloc[-1]) * 100:.2f}%")

# Demonstrate gap detection
print(f"\n" + "=" * 70)
print("Why ATR > Simple Range:")
print(f"\nTR considers:")
print(f"  1. High - Low = {(tg_high.iloc[-1] - tg_low.iloc[-1]):.4f}")
print(f"  2. |High - Prev Close| = {abs(tg_high.iloc[-1] - tg_close.iloc[-2]):.4f}")
print(f"  3. |Low - Prev Close| = {abs(tg_low.iloc[-1] - tg_close.iloc[-2]):.4f}")
print(f"\nTR = max of above = {tr.iloc[-1]:.4f}")
print(f"\nThis captures gaps that simple range misses!")

In [ ]:
# Visualize ATR vs Simple Range
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Price
axes[0].plot(tg_close.index, tg_close.values,
            linewidth=1.5, color='black', alpha=0.7)
axes[0].set_title('Top Glove Price', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Price (RM)')
axes[0].grid(True, alpha=0.3)

# ATR vs Simple Range
axes[1].plot(atr.index, atr.values,
            linewidth=2, label='ATR (14)', color='red', alpha=0.9)
axes[1].plot(simple_range.rolling(14).mean().index,
            simple_range.rolling(14).mean().values,
            linewidth=2, label='Simple Range (14-MA)', color='blue',
            linestyle='--', alpha=0.7)
axes[1].fill_between(atr.index, atr.values, alpha=0.2, color='red')
axes[1].set_title('ATR vs Simple Range Comparison', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Range (RM)')
axes[1].legend(loc='best')
axes[1].grid(True, alpha=0.3)

# ATR as percentage of price (normalized volatility)
atr_pct = (atr / tg_close) * 100
axes[2].plot(atr_pct.index, atr_pct.values,
            linewidth=2, color='purple', alpha=0.8)
axes[2].axhline(y=atr_pct.mean(), color='orange', linestyle='--',
               linewidth=2, label=f'Average ({atr_pct.mean():.2f}%)')
axes[2].fill_between(atr_pct.index, 0, atr_pct.values,
                     where=(atr_pct > atr_pct.quantile(0.75)),
                     alpha=0.3, color='red', label='High volatility')
axes[2].fill_between(atr_pct.index, 0, atr_pct.values,
                     where=(atr_pct < atr_pct.quantile(0.25)),
                     alpha=0.3, color='green', label='Low volatility')
axes[2].set_title('ATR as % of Price (Normalized Volatility)', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Date')
axes[2].set_ylabel('ATR %')
axes[2].legend(loc='best')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nATR Statistics:")
print(f"  Average ATR: RM {atr.mean():.4f}")
print(f"  Average ATR %: {atr_pct.mean():.2f}%")
print(f"  Min ATR: RM {atr.min():.4f} (low volatility)")
print(f"  Max ATR: RM {atr.max():.4f} (high volatility)")
print(f"\nInterpretation:")
print(f"  • ATR captures ACTUAL price movement including gaps")
print(f"  • Higher ATR = Higher volatility = More risk")
print(f"  • Use ATR % for comparing different stocks")

### Using ATR for Stop-Loss Placement

**Adaptive Stop-Loss Formula:**

$$
\text{Stop Distance} = k \times ATR
$$

Where:
- $k$ = Multiplier (typically 1.5 to 3)
- Higher k = Wider stop (more room for noise)
- Lower k = Tighter stop (less risk but more whipsaws)

**Long position:**
$$
\text{Stop Loss} = \text{Entry Price} - (k \times ATR)
$$

**Short position:**
$$
\text{Stop Loss} = \text{Entry Price} + (k \times ATR)
$$

In [ ]:
# Demonstrate ATR-based stop-loss
entry_price = tg_close.iloc[-1]
current_atr = atr.iloc[-1]

print("ATR-Based Stop-Loss Calculation\n")
print("=" * 70)
print(f"\nAssuming LONG entry at: RM {entry_price:.4f}")
print(f"Current ATR: RM {current_atr:.4f}\n")

multipliers = [1.5, 2, 2.5, 3]
print(f"{'Multiplier':>10} | {'Stop Loss':>12} | {'Risk':>10} | {'Risk %':>8}")
print("-" * 70)

for k in multipliers:
    stop_loss = entry_price - (k * current_atr)
    risk_amount = entry_price - stop_loss
    risk_pct = (risk_amount / entry_price) * 100
    
    print(f"{k:10.1f} | RM {stop_loss:10.4f} | RM {risk_amount:8.4f} | {risk_pct:7.2f}%")

print("\n" + "=" * 70)
print("Recommendation:")
print("  • Conservative: 3× ATR (less whipsaws)")
print("  • Moderate: 2× ATR (balanced)")
print("  • Aggressive: 1.5× ATR (tight stop)")
print("\nATR adapts to volatility automatically!")

---

## Part 3: Keltner Channels - ATR-Based Bands

### What are Keltner Channels?

**Keltner Channels** are similar to Bollinger Bands but use ATR instead of standard deviation.

### Formula

$$
\begin{aligned}
\text{Middle Line} &= EMA_n(\text{Close}) \\
\text{Upper Channel} &= EMA_n + (k \times ATR) \\
\text{Lower Channel} &= EMA_n - (k \times ATR)
\end{aligned}
$$

Typical settings:
- EMA period: 20
- ATR period: 10 or 14
- Multiplier: 2

### Keltner vs Bollinger Bands

| Feature | Bollinger Bands | Keltner Channels |
|---------|----------------|------------------|
| Center | SMA | EMA |
| Width | Standard Deviation | ATR |
| Adapts to | Price spread | Actual range + gaps |
| Best for | Mean reversion | Trend following |
| Sensitivity | More responsive | Smoother |

In [ ]:
def calculate_keltner_channels(high, low, close, ema_period=20, atr_period=10, multiplier=2):
    """
    Calculate Keltner Channels.
    
    Parameters:
    -----------
    high, low, close : pd.Series
        Price data
    ema_period : int
        EMA period for center line (default 20)
    atr_period : int
        ATR period (default 10)
    multiplier : float
        ATR multiplier (default 2)
    
    Returns:
    --------
    tuple: (middle, upper, lower)
    """
    # Calculate middle line (EMA)
    middle = close.ewm(span=ema_period, adjust=False).mean()
    
    # Calculate ATR
    _, atr_values = calculate_atr(high, low, close, period=atr_period)
    
    # Calculate channels
    upper = middle + (multiplier * atr_values)
    lower = middle - (multiplier * atr_values)
    
    return middle, upper, lower

# Calculate Keltner Channels for CIMB
cimb_high = cimb['High']
cimb_low = cimb['Low']
cimb_close = cimb['Close']

kc_middle, kc_upper, kc_lower = calculate_keltner_channels(
    cimb_high, cimb_low, cimb_close, ema_period=20, atr_period=10, multiplier=2
)

# Also calculate Bollinger Bands for comparison
bb_middle_cimb, bb_upper_cimb, bb_lower_cimb, _, _ = calculate_bollinger_bands(
    cimb_close, period=20, num_std=2
)

print("Keltner Channels vs Bollinger Bands (CIMB)\n")
print("=" * 70)
print(f"\nCurrent Price: RM {cimb_close.iloc[-1]:.4f}")
print(f"\nKeltner Channels (20, 10, 2):")
print(f"  Upper: RM {kc_upper.iloc[-1]:.4f}")
print(f"  Middle: RM {kc_middle.iloc[-1]:.4f}")
print(f"  Lower: RM {kc_lower.iloc[-1]:.4f}")
print(f"  Width: RM {kc_upper.iloc[-1] - kc_lower.iloc[-1]:.4f}")

print(f"\nBollinger Bands (20, 2):")
print(f"  Upper: RM {bb_upper_cimb.iloc[-1]:.4f}")
print(f"  Middle: RM {bb_middle_cimb.iloc[-1]:.4f}")
print(f"  Lower: RM {bb_lower_cimb.iloc[-1]:.4f}")
print(f"  Width: RM {bb_upper_cimb.iloc[-1] - bb_lower_cimb.iloc[-1]:.4f}")

In [ ]:
# Visualize Keltner vs Bollinger
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Bollinger Bands
axes[0].plot(cimb_close.index, cimb_close.values,
            linewidth=1.5, label='Price', color='black', alpha=0.8, zorder=5)
axes[0].plot(bb_middle_cimb.index, bb_middle_cimb.values,
            linewidth=2, label='BB Middle (SMA 20)', color='blue', linestyle='--')
axes[0].plot(bb_upper_cimb.index, bb_upper_cimb.values,
            linewidth=1.5, label='BB Upper', color='red', linestyle=':', alpha=0.7)
axes[0].plot(bb_lower_cimb.index, bb_lower_cimb.values,
            linewidth=1.5, label='BB Lower', color='green', linestyle=':', alpha=0.7)
axes[0].fill_between(bb_middle_cimb.index, bb_lower_cimb, bb_upper_cimb,
                     alpha=0.1, color='gray')
axes[0].set_title('CIMB - Bollinger Bands (20, 2)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Price (RM)')
axes[0].legend(loc='best', fontsize=9)
axes[0].grid(True, alpha=0.3)

# Keltner Channels
axes[1].plot(cimb_close.index, cimb_close.values,
            linewidth=1.5, label='Price', color='black', alpha=0.8, zorder=5)
axes[1].plot(kc_middle.index, kc_middle.values,
            linewidth=2, label='KC Middle (EMA 20)', color='purple', linestyle='--')
axes[1].plot(kc_upper.index, kc_upper.values,
            linewidth=1.5, label='KC Upper', color='red', linestyle=':', alpha=0.7)
axes[1].plot(kc_lower.index, kc_lower.values,
            linewidth=1.5, label='KC Lower', color='green', linestyle=':', alpha=0.7)
axes[1].fill_between(kc_middle.index, kc_lower, kc_upper,
                     alpha=0.1, color='purple')
axes[1].set_title('CIMB - Keltner Channels (20, 10, 2)', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Price (RM)')
axes[1].legend(loc='best', fontsize=9)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Differences:")
print("  Bollinger Bands:")
print("    • Uses SMA (equal weight)")
print("    • Width based on standard deviation")
print("    • More volatile, reactive to spikes")
print("    • Better for mean reversion")

print("\n  Keltner Channels:")
print("    • Uses EMA (exponential weight)")
print("    • Width based on ATR")
print("    • Smoother, less false signals")
print("    • Better for trend following")

### The Squeeze: Combining Bollinger & Keltner

**Volatility Squeeze Pattern:**

1. **Squeeze = Bollinger Bands inside Keltner Channels**
   - Low volatility period
   - Consolidation
   - Breakout likely coming

2. **Release = Bollinger Bands expand outside Keltner**
   - Volatility explosion
   - Strong directional move
   - Trade in direction of breakout

This is a powerful pattern for breakout trading!

In [ ]:
# Detect squeeze conditions
# Squeeze = BB inside KC (BB upper < KC upper AND BB lower > KC lower)
squeeze = (bb_upper_cimb < kc_upper) & (bb_lower_cimb > kc_lower)

# Calculate squeeze strength
bb_width_cimb = bb_upper_cimb - bb_lower_cimb
kc_width = kc_upper - kc_lower
squeeze_ratio = bb_width_cimb / kc_width

# Visualize squeeze
plt.figure(figsize=(14, 10))

# Price with both indicators
plt.subplot(2, 1, 1)
plt.plot(cimb_close.index, cimb_close.values,
        linewidth=1.5, label='Price', color='black', alpha=0.8, zorder=5)

# Bollinger Bands
plt.plot(bb_upper_cimb.index, bb_upper_cimb.values,
        linewidth=1.5, label='BB Upper', color='red', linestyle=':', alpha=0.6)
plt.plot(bb_lower_cimb.index, bb_lower_cimb.values,
        linewidth=1.5, label='BB Lower', color='green', linestyle=':', alpha=0.6)

# Keltner Channels
plt.plot(kc_upper.index, kc_upper.values,
        linewidth=2, label='KC Upper', color='red', linestyle='--', alpha=0.8)
plt.plot(kc_lower.index, kc_lower.values,
        linewidth=2, label='KC Lower', color='green', linestyle='--', alpha=0.8)

# Highlight squeeze periods
for i in range(len(squeeze)):
    if squeeze.iloc[i]:
        plt.axvspan(squeeze.index[i], squeeze.index[min(i+1, len(squeeze)-1)],
                   alpha=0.2, color='yellow')

plt.title('The Squeeze: Bollinger Bands vs Keltner Channels (Yellow = Squeeze)',
         fontsize=12, fontweight='bold')
plt.ylabel('Price (RM)')
plt.legend(loc='best', fontsize=9)
plt.grid(True, alpha=0.3)

# Squeeze indicator
plt.subplot(2, 1, 2)
colors = ['yellow' if s else 'gray' for s in squeeze]
plt.bar(squeeze_ratio.index, squeeze_ratio.values, color=colors, alpha=0.6, width=1)
plt.axhline(y=1, color='red', linestyle='--', linewidth=2,
           label='Threshold (BB width = KC width)')
plt.title('Squeeze Ratio (BB Width / KC Width)', fontsize=12, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Ratio')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistics
print(f"\nSqueeze Analysis:")
print(f"  Total squeeze days: {squeeze.sum()} ({squeeze.sum()/len(squeeze)*100:.1f}%)")
print(f"  Average squeeze ratio: {squeeze_ratio[squeeze].mean():.4f}")
print(f"\nTrading Strategy:")
print(f"  1. Wait for squeeze (yellow zones)")
print(f"  2. Watch for BB to break outside KC")
print(f"  3. Trade in direction of breakout")
print(f"  4. Use ATR for stop-loss")

---

## Part 4: Position Sizing with ATR

### Risk-Based Position Sizing

**Formula:**

$$
\text{Position Size} = \frac{\text{Account Risk Amount}}{\text{ATR} \times k}
$$

Where:
- Account Risk Amount = Capital × Risk % (e.g., RM 10,000 × 2% = RM 200)
- ATR = Current Average True Range
- k = Stop-loss multiplier (e.g., 2)

**Example:**
- Account: RM 10,000
- Risk per trade: 2% = RM 200
- Stock price: RM 5.00
- ATR: RM 0.25
- Stop multiplier: 2 (stop at price - 2×ATR)

$$
\text{Position Size} = \frac{200}{0.25 \times 2} = \frac{200}{0.50} = 400 \text{ shares}
$$

In [ ]:
def calculate_position_size(account_size, risk_pct, price, atr_value, stop_multiplier=2):
    """
    Calculate position size based on ATR risk.
    
    Parameters:
    -----------
    account_size : float
        Total account value (RM)
    risk_pct : float
        Risk percentage per trade (e.g., 2.0 for 2%)
    price : float
        Entry price (RM)
    atr_value : float
        Current ATR (RM)
    stop_multiplier : float
        ATR multiplier for stop-loss (default 2)
    
    Returns:
    --------
    dict: Position sizing details
    """
    # Calculate risk amount
    risk_amount = account_size * (risk_pct / 100)
    
    # Calculate stop distance
    stop_distance = atr_value * stop_multiplier
    
    # Calculate position size (shares)
    position_size = risk_amount / stop_distance
    
    # Calculate stop-loss price (for long)
    stop_loss = price - stop_distance
    
    # Calculate position value
    position_value = position_size * price
    
    return {
        'account_size': account_size,
        'risk_pct': risk_pct,
        'risk_amount': risk_amount,
        'price': price,
        'atr': atr_value,
        'stop_multiplier': stop_multiplier,
        'stop_distance': stop_distance,
        'stop_loss': stop_loss,
        'position_size': position_size,
        'position_value': position_value
    }

# Example: Position sizing for Top Glove
account = 10000  # RM 10,000
risk = 2.0  # 2% risk per trade
current_price = tg_close.iloc[-1]
current_atr_tg = atr.iloc[-1]

sizing = calculate_position_size(account, risk, current_price, current_atr_tg, stop_multiplier=2)

print("ATR-Based Position Sizing Example\n")
print("=" * 70)
print(f"\nAccount Details:")
print(f"  Account size: RM {sizing['account_size']:,.2f}")
print(f"  Risk per trade: {sizing['risk_pct']:.1f}%")
print(f"  Risk amount: RM {sizing['risk_amount']:.2f}")

print(f"\nStock: Top Glove (5225.KL)")
print(f"  Entry price: RM {sizing['price']:.4f}")
print(f"  Current ATR: RM {sizing['atr']:.4f}")
print(f"  Stop multiplier: {sizing['stop_multiplier']:.1f}×")

print(f"\nCalculated Position:")
print(f"  Stop distance: RM {sizing['stop_distance']:.4f}")
print(f"  Stop-loss price: RM {sizing['stop_loss']:.4f}")
print(f"  Position size: {sizing['position_size']:.0f} shares")
print(f"  Position value: RM {sizing['position_value']:,.2f}")

print(f"\nVerification:")
print(f"  If stopped out at RM {sizing['stop_loss']:.4f}:")
print(f"  Loss per share: RM {sizing['stop_distance']:.4f}")
print(f"  Total loss: {sizing['position_size']:.0f} × RM {sizing['stop_distance']:.4f} = RM {sizing['position_size'] * sizing['stop_distance']:.2f}")
print(f"  This equals our risk amount: RM {sizing['risk_amount']:.2f} ✓")

In [ ]:
# Compare position sizes for different volatility stocks
print("Position Sizing Comparison Across Stocks\n")
print("=" * 80)
print(f"\nAccount: RM 10,000 | Risk: 2% per trade = RM 200")
print(f"Stop: 2× ATR\n")

# Calculate for all three stocks
stocks = [
    ('Maybank', maybank['Close'].iloc[-1], calculate_atr(maybank['High'], maybank['Low'], maybank['Close'])[1].iloc[-1]),
    ('Top Glove', tg_close.iloc[-1], atr.iloc[-1]),
    ('CIMB', cimb_close.iloc[-1], calculate_atr(cimb['High'], cimb['Low'], cimb['Close'])[1].iloc[-1])
]

print(f"{'Stock':<12} | {'Price':>8} | {'ATR':>8} | {'Stop':>8} | {'Shares':>8} | {'Value':>10}")
print("-" * 80)

for stock_name, price, atr_val in stocks:
    sizing = calculate_position_size(10000, 2.0, price, atr_val, 2)
    print(f"{stock_name:<12} | RM {price:6.2f} | RM {atr_val:5.3f} | RM {sizing['stop_loss']:5.2f} | {sizing['position_size']:6.0f} | RM {sizing['position_value']:8,.2f}")

print("\n" + "=" * 80)
print("Key Insights:")
print("  • Higher volatility (ATR) → Fewer shares (to maintain same risk)")
print("  • Lower volatility → More shares allowed")
print("  • All positions risk exactly RM 200 if stopped out")
print("  • Position size automatically adapts to volatility!")

---

## Part 5: Exercises

Practice volatility analysis with these exercises.

### Exercise 1: Bollinger Band Squeeze Scanner

Create a scanner to find the tightest BB squeezes.

**Tasks**:
1. Calculate bandwidth for all three stocks
2. Normalize bandwidth (as % of price)
3. Identify which stock has the tightest squeeze currently
4. Show historical squeeze patterns
5. Predict when expansion is likely

In [ ]:
# Your code here



### Exercise 2: ATR Trailing Stop System

Build a trailing stop-loss system using ATR.

**Tasks**:
1. Implement ATR-based trailing stop (e.g., 3× ATR below price)
2. Backtest on 2023 Maybank data
3. Calculate max drawdown with this stop
4. Compare with fixed % stop (e.g., 5%)
5. Visualize both stop methods

In [ ]:
# Your code here



### Exercise 3: Volatility Regime Detection

Classify market into low/medium/high volatility regimes.

**Tasks**:
1. Calculate ATR percentile (where is current ATR vs historical?)
2. Define regimes: Low (<25th percentile), Medium (25-75), High (>75th)
3. Count days in each regime for 2023
4. Analyze returns in different regimes
5. Recommend strategies for each regime

In [ ]:
# Your code here



### Exercise 4: Multi-Timeframe Volatility Analysis

Analyze volatility across different timeframes.

**Tasks**:
1. Calculate daily, weekly, and monthly ATR
2. Compare volatility at each timeframe
3. Identify timeframe expansion/contraction
4. Create a volatility dashboard
5. Use for position sizing across timeframes

In [ ]:
# Your code here



---

## Summary

Congratulations! You've completed Module 06. Let's review:

### Key Concepts Mastered

1. **Bollinger Bands**
   - Formula: SMA ± (k × σ)
   - Statistical foundation (68-95-99.7 rule)
   - Bandwidth for volatility measurement
   - %B for position within bands
   - Squeeze detection

2. **Average True Range (ATR)**
   - True Range = max(High-Low, |High-PrevClose|, |Low-PrevClose|)
   - Captures gaps that simple range misses
   - Wilder's smoothing (EMA with α = 1/14)
   - Pure volatility measure

3. **Keltner Channels**
   - Formula: EMA ± (k × ATR)
   - Smoother than Bollinger Bands
   - Better for trend following
   - Uses ATR instead of σ

4. **The Squeeze**
   - Bollinger Bands inside Keltner = Low volatility
   - Breakout indicator
   - High-probability setup

5. **Practical Applications**
   - ATR-based stop-loss placement
   - Position sizing with volatility
   - Volatility regime detection
   - Risk management

### Mathematical Insights

- **Bollinger Bands** = Statistical (based on standard deviation)
- **Keltner Channels** = Range-based (based on actual movement)
- **ATR** captures true volatility including gaps
- Position sizing inversely proportional to volatility

### What's Next?

**Module 07: Trend and Convergence Mathematics** will cover:
- MACD deep dive (complete calculation)
- ADX (Average Directional Index)
- Parabolic SAR
- Trend strength measurement

---

## Additional Resources

### Further Reading
- Bollinger, John (2001). "Bollinger on Bollinger Bands"
- Wilder, J. Welles (1978). "New Concepts in Technical Trading Systems" (ATR)
- [Investopedia: Bollinger Bands](https://www.investopedia.com/terms/b/bollingerbands.asp)
- [Investopedia: ATR](https://www.investopedia.com/terms/a/atr.asp)
- [Investopedia: Keltner Channels](https://www.investopedia.com/terms/k/keltnerchannel.asp)

---

**Excellent work!** You now understand how to measure, analyze, and trade volatility. Ready for Module 07?